In [1]:
import pandas as pd

from core import extract_embeddings, load_sentence_model, semantic_search
from core import load_actions_dataset, compute_effects_from_indicator, generate_policies_from_bills

In [2]:
# 1) Carregue o dataset de PLs/ações (precisa ter 'municipio', 'data_apresentacao', 'acao')
dataset = load_actions_dataset("data/dataset.npy")

emb = extract_embeddings(dataset)
model = load_sentence_model()

# 2) Carregue o indicador (mesma normalização de nomes de cidade usada no indicador)
indicator = pd.read_csv("data/criminal_indicator.csv")  # deve conter municipio_norm, ano, semestre, taxa_homicidios_100k

In [26]:
matches = semantic_search(
    query="Como reduzir a criminalidade no município?", 
    dataset=dataset, model=model, embeddings=emb, top_k=500
)

In [27]:
# 3) Compute os efeitos (delta entre semestre atual e próximo)
effects = compute_effects_from_indicator(
    bills=matches,
    indicator_df=indicator,
    city_col="municipio_norm",
    value_col="taxa_homicidios_100k",
)

In [29]:
import numpy as np

# effects é uma lista de (municipio, descricao_do_pl, delta_indicador)
policies = generate_policies_from_bills(effects, min_group_members=3, similarity_threshold=0.75)

# 4) Ordene e inspecione
policies = sorted(policies, key=lambda x: x['quality_score'], reverse=True)

for p in policies:
    scores = np.array([row[2] for row in p['actions']])
    n =  scores.shape[0]
    wins = np.sum(scores < 0.0)

    if (wins / n) <= 0.5:
        continue

    print(f"(Score: {p['quality_score']:.3f} - Win Rate: {wins}/{n}) {p['policy']}")


(Score: 6.870 - Win Rate: 4/4) Implantar o programa “Bueiro Imobiliário” para prevenção de enchentes no município.
(Score: 5.325 - Win Rate: 5/6) Criar programa de combate a pichações no município.
(Score: 4.975 - Win Rate: 3/3) Criar o programa “Cuidando de Quem Cuida” no município.
(Score: 4.637 - Win Rate: 3/4) Criar câmara mirim no município.
(Score: 3.017 - Win Rate: 3/3) Instituir a campanha “Janeiro Branco” no município.
(Score: 2.089 - Win Rate: 4/5) Instituir o mês “Abril Laranja” como mês de prevenção da crueldade contra os animais no município.
(Score: 0.932 - Win Rate: 3/5) Proibir blitz do IPVA no município.
(Score: 0.000 - Win Rate: 2/3) Instituir o “Junho Violeta” no município.
